# Ejercicio "Transfer Learning" con Inception V3 y ResNet50

En este cuaderno se utiliza la red InceptionV3 de Google, preentrenada con el conjunto de datos Imagenet.

Primero le añadimos un cabezal clasificador adaptado a nuestro conjunto de datos y sólo entrenamos este cabezal.

Después vemos como modificar también alguas capas de la red preentrenada.

Finalmente se propone hacer lo mismo con ResNet50.

---

    [ES] Código de Alfredo Cuesta Infante para 'Reconocimiento de Patrones'
       @ Master Universitario en Visión Artificial, 2020, URJC (España)
    [EN] Code by Alfredo Cuesta-Infante for 'Pattern Recognition'
       @ Master of Computer Vision, 2020, URJC (Spain)

    alfredo.cuesta@urjc.es    

**Módulos básicos**

In [1]:
import numpy as np
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

**Crear la "base" del modelo *Inception* pre-entrenado con ImageNet**

¡¡ Observa que  `include_top = False` para poder añadirle nuestro cabezal clasificador !!

In [2]:
base_model = InceptionV3(weights='imagenet', include_top=False) 

### 1. Añadir un cabezal clasificador a la red preentrenada

**Añadir 2 nuevas capas**
- Vamos a suponer que hay 200 clases en nuestro nuevo problema

In [3]:
# Sobre el modelo base, primero hacemos un agrupamiento por promedio
#  (puede ser otro agrupamiento, he elegido este para usar uno diferente del típico MaxPool)
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Luego añadimos una capa densa, oculta, de 1024 neuronas
x = Dense(1024, activation='relu')(x)

# Y finalmente una capa con activación Softmax para clasificar entre las 200 
#  clases posibles. 
predictions = Dense(200, activation='softmax')(x)

# El modelo final queda:
model = Model(inputs=base_model.input, outputs=predictions)

In [4]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, None, None, 3 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, None, None, 3 0           batch_normalization[0][0]        
_______________________________________________________________________________________

**Entrenamiento**

+ Vamos a entrenar **SÓLO las dos capas nuevas**, es decir 'congelamos' las capas de Inception V3.

In [5]:
# Recorremos cada una de las capas de base_model desactivando el bit 'trainable'
for layer in base_model.layers:
    layer.trainable = False

# Compilamos el modelo con un optimizador, una pérdida, ...
model.compile(optimizer='rmsprop', \
              loss='categorical_crossentropy')

In [ ]:
# Y lanzamos el método FIT para aprender con los datos dados 
model.fit(...)

$\rightarrow$ Como resultado, las 2 nuevas capas han quedado aprendidas, es decir hemos ajustado sus pesos.

### 2. Modificar la red preentrenada, actualizando los pesos de sus últimas capas.

In [6]:
# Primero mostramos en pantalla el nombre de cada capa
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

# Vamos a elegir reentrenar las dos últimas capas de InceptionV3,
#   para lo cual hay que 'congelar' las primeras 249 capas
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True

# Recompilamos el modelo. 
# Podemos usar otro optimizador, incluso personalizarlo con opciones como 
#    learning rate (lr) o 'inercia' (momentum)
from tensorflow.keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), \
              loss='categorical_crossentropy')

0 input_1
1 conv2d
2 batch_normalization
3 activation
4 conv2d_1
5 batch_normalization_1
6 activation_1
7 conv2d_2
8 batch_normalization_2
9 activation_2
10 max_pooling2d
11 conv2d_3
12 batch_normalization_3
13 activation_3
14 conv2d_4
15 batch_normalization_4
16 activation_4
17 max_pooling2d_1
18 conv2d_8
19 batch_normalization_8
20 activation_8
21 conv2d_6
22 conv2d_9
23 batch_normalization_6
24 batch_normalization_9
25 activation_6
26 activation_9
27 average_pooling2d
28 conv2d_5
29 conv2d_7
30 conv2d_10
31 conv2d_11
32 batch_normalization_5
33 batch_normalization_7
34 batch_normalization_10
35 batch_normalization_11
36 activation_5
37 activation_7
38 activation_10
39 activation_11
40 mixed0
41 conv2d_15
42 batch_normalization_15
43 activation_15
44 conv2d_13
45 conv2d_16
46 batch_normalization_13
47 batch_normalization_16
48 activation_13
49 activation_16
50 average_pooling2d_1
51 conv2d_12
52 conv2d_14
53 conv2d_17
54 conv2d_18
55 batch_normalization_12
56 batch_normalization_14
5

In [ ]:
# Y lanzamos el método FIT para aprender con los datos dados 
model.fit_generator(...)

## 3. ResNet50

Trata de repetir el ejercicio con la red ResNet50

AYUDA:
>   from tensorflow.keras.applications.resnet50 import ResNet50  <br>
>   model = ResNet50(weights='imagenet')